In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_fp = '../input/tabular-playground-series-aug-2021/train.csv'
train = pd.read_csv(train_fp)

test_fp = '../input/tabular-playground-series-aug-2021/test.csv'
test = pd.read_csv(test_fp)

In [ ]:
train

# EDA

In [ ]:
#plt.figure(figsize=(10, 6))
sns.displot(train['loss'], kde=True, height=6, aspect=2)

In [ ]:
train_corr = train.corr()
mask = np.triu(np.ones_like(train_corr, dtype=np.bool))

fig = plt.figure(figsize=(16,13))
sns.heatmap(train_corr, mask=mask)
plt.title('Correlation between features')

In [ ]:
X = train.drop(['loss'], axis=1)
y = train.loss

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.3)

In [ ]:
'''
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

xgbr = XGBRegressor(seed=20)

params = { "max_depth": [2, 3, 4],
           "learning_rate": [0.03, 0.04, 0.05],
           "n_estimators": np.arange(100, 1000, 100),
           "colsample_bytree": np.arange(0.2, 0.7, 0.1),
            "colsample_bylevel": np.arange(0.2, 0.7, 0.1),
            "colsample_bynode": np.arange(0.2, 0.7, 0.1)
            }

reg = RandomizedSearchCV(estimator=xgbr,
                  param_distributions=params,
                  scoring='neg_root_mean_squared_error',
                    n_iter=100,
                  verbose=1)
'''

In [ ]:
'''
reg.fit(X_train, y_train)

print("Best parameters:", reg.best_params_)
'''

In [ ]:

from xgboost import XGBRegressor

params = {'n_estimators': 900,
         'learning_rate': 0.05,
         'max_depth': 4,
         'colsample_bytree': 0.5000000000000001,
         'colsample_bynode': 0.2,
         'colsample_bylevel': 0.4000000000000001}

model = XGBRegressor(**params)

model.fit(X_train, y_train,
         early_stopping_rounds=5,
         eval_set=[(X_val, y_val)],
         verbose=False)

preds = model.predict(X_val)

In [ ]:

from sklearn.metrics import accuracy_score, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score 

mae = "Mean absolute error: {}".format(mean_absolute_error(y_val, preds))
print(mae)

r2 = "r2 score: {}".format(r2_score(y_val, preds))
print(r2)


In [ ]:
test

In [ ]:
test_trimmed = test[:150000]

In [ ]:
final_output = model.predict(test_trimmed)

In [ ]:
final_output

In [ ]:
'''
final_mae = "Mean absolute error: {}".format(mean_absolute_error(y_val, final_output))
print(mae)

final_r2 = "r2 score: {}".format(r2_score(y_val, final_output))
print(r2)
'''

In [ ]:
sample_fp = "../input/tabular-playground-series-aug-2021/sample_submission.csv"
sample = pd.read_csv(sample_fp)

In [ ]:
sample_trimmed = sample.copy()
sample_trimmed = sample_trimmed[:150000]
sample_trimmed['loss'] = final_output

sample_trimmed.to_csv('submission.csv', index=False)

In [ ]:
sample_trimmed